**Libraries Used**

In [1]:
import nltk  # Import NLTK for natural language processing utilities
import json  # Import JSON for parsing structured data
import numpy as np  # Import NumPy for numerical operations
import pickle as pk  # Import pickle (aliased as pk) for loading serialized Python objects
import random as rnd  # Import random module (aliased as rnd) for random response selection
import tkinter as tk  # Import tkinter with alias for GUI creation
from tkinter import *  # Import all tkinter components for direct widget access
from keras.models import load_model  # Import Keras function to load a trained model
Model = load_model('chatbot_model_01.h5')  # Load the previously trained chatbot model from file
from nltk.stem import WordNetLemmatizer  # Import WordNetLemmatizer for word normalization
lemmatizer = WordNetLemmatizer()  # Create a lemmatizer instance for preprocessing input text

**Intent Prediction and Response Generation**

In [2]:
def clean_up_sentence(sentence):  # Normalize and tokenize the input sentence
    sentence_words = nltk.word_tokenize(sentence)  # Split sentence into word tokens
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]  # Lowercase and lemmatize each token
    return sentence_words  # Return cleaned list of words

def bag_of_words(sentence, words, show_details=True):  # Convert sentence into bag-of-words vector
    sentence_words = clean_up_sentence(sentence)  # Get cleaned tokens from input sentence
    bag = [0] * len(words)  # Initialize zero vector with vocabulary length
    for s in sentence_words:  # Loop through each token in the sentence
        for i, word in enumerate(words):  # Compare token with each vocabulary word
            if word == s:  # If token exists in vocabulary
                bag[i] = 1  # Mark corresponding index as present
                if show_details:  # If debug details are enabled
                    print("found in bag: %s" % word)  # Print matched word
    return np.array(bag)  # Return bag-of-words as NumPy array

def predict_class(sentence):  # Predict likely intent classes for a sentence
    p = bag_of_words(sentence, words, show_details=False)  # Build input vector without debug prints
    res = Model.predict(np.array([p]))[0]  # Run model prediction and get probabilities
    ERROR_THRESHOLD = 0.25  # Minimum confidence threshold for accepted intents
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]  # Keep intents above threshold
    results.sort(key=lambda x: x[1], reverse=True)  # Sort intents by confidence descending
    return_list = []  # Initialize list for formatted predictions
    for r in results:  # Iterate over filtered predictions
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})  # Store intent label and probability
    return return_list  # Return ranked predicted intents

def get_response(ints, intents_json):  # Select a response based on predicted intent
    tag = ints[0]['intent']  # Get top predicted intent tag
    list_of_intents = intents_json['intents']  # Access list of all intents
    for i in list_of_intents:  # Loop through intents to find matching tag
        if i['tag'] == tag:  # If current intent matches predicted tag
            result = rnd.choice(i['responses'])  # Choose a random response from that intent
            break  # Stop loop after finding the first match
    return result  # Return selected chatbot response

**Chatbot GUI and Message Handling**

In [3]:
intants = json.loads(open('intents.json' , encoding='utf-8').read())  # Load intents data from JSON file
words = pk.load(open('words.pkl', 'rb'))  # Load saved vocabulary list
classes = pk.load(open('classes.pkl', 'rb'))  # Load saved class labels

def send():  # Handle user message send action
    msg = EntryBox.get("1.0", 'end-1c').strip()  # Read text from input box and trim spaces
    EntryBox.delete("0.0", END)  # Clear the input box after reading message

    if msg != '':  # Continue only if message is not empty
        ChatLog.config(state=NORMAL)  # Enable chat log for writing
        ChatLog.insert(END, "You: " + msg + '\n\n')  # Show user message in chat log
        ChatLog.config(foreground="#442265", font=("Verdana", 12))  # Set chat log text style

        tag_map = {i['tag'].lower(): i['responses'] for i in intants['intents']}  # Build map: tag -> responses
        msg_l = msg.lower().strip()  # Normalize message to lowercase
        words_in_msg = set(nltk.word_tokenize(msg_l))  # Tokenize message words into a set

        matched_tag = None  # Initialize matched tag as not found
        if msg_l in tag_map:  # If full message exactly matches a tag
            matched_tag = msg_l  # Use that tag directly
        else:
            for t in tag_map.keys():  # Otherwise check if any tag appears in message words
                if t in words_in_msg:  # If a tag word is found in tokenized message
                    matched_tag = t  # Set matched tag
                    break  # Stop after first match

        if matched_tag:  # If a direct/simple tag match was found
            res = rnd.choice(tag_map[matched_tag])  # Pick random response from matched tag
        else:
            ints = predict_class(msg)  # Predict intent with model
            res = get_response(ints, intants)  # Get response based on predicted intent

        ChatLog.insert(END, "Bot: " + res + '\n\n')  # Show bot response in chat log
        ChatLog.config(state=DISABLED)  # Lock chat log to prevent manual edits
        ChatLog.yview(END)  # Auto-scroll to latest message

root = tk.Tk()  # Create main application window
root.title("ChatBot")  # Set window title
root.geometry("400x500")  # Set initial window size

root.resizable(width=True, height=True)  # Allow resizing in both directions
ChatLog = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)  # Create chat display area
ChatLog.config(state=DISABLED)  # Start with chat display read-only
scrollbar = Scrollbar(root, command=ChatLog.yview, cursor="heart")  # Create scrollbar linked to chat log
ChatLog['yscrollcommand'] = scrollbar.set  # Sync chat log scrolling with scrollbar

sendButton = Button(root, font=("Verdana", 12, 'bold'), text="Send", width="12", height=5,  # Create send button
                    bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff', command=send)  # Style button and bind click event

EntryBox = Text(root, bd=0, bg="white", width="29", height="5", font="Arial")  # Create text input box for user messages
scrollbar.place(x=376, y=6, height=386)  # Position scrollbar in window
ChatLog.place(x=6, y=6, height=386, width=370)  # Position chat log area
EntryBox.place(x=128, y=401, height=90, width=265)  # Position input box
sendButton.place(x=6, y=401, height=90)  # Position send button
root.mainloop()  # Start Tkinter event loop

1/1 [==============================] - 0s 34ms/step
